In [2]:
import pandas as pd
import os
import json
import numpy as np

In [3]:
with open("../siqa_task.json", mode="r") as f:
    siqa_task = json.load(f)

siqa_data = siqa_task['examples']

In [10]:
# labels_df = pd.DataFrame(columns=['idx', 'sentence', 'label'])
idx = []
sentences = []
targets = []
for i, item in enumerate(siqa_data):
    s = item['input']
    # print(list(item['target_scores'].values()))
    label = np.argmax(list(item['target_scores'].values()))
    idx.append(i)
    sentences.append(s)
    targets.append(label)

labels_df = pd.DataFrame({'idx': idx, 'sentences': sentences, "targets": targets})
labels_df

,idx,sentences,targets
0,0,Tracy didn't go home that evening and resisted...,2
1,1,Sydney walked past a homeless woman asking for...,0
2,2,Sasha protected the patients' rights by making...,1
3,3,Jordan was in charge of taking the food on the...,0
4,4,Kendall opened their mouth to speak and what c...,2
...,...,...,...
1949,1949,Bailey destroyed the enemy's bunker and won th...,0
1950,1950,Kai told Kendall the reason about the accident...,1
1951,1951,Kai improved Jan's picture and she loved how i...,2
1952,1952,Tracy's kids wanted ice cream so Aubrey fed th...,1


In [4]:
base_csv_pth = "../baselines"
male_csv = "0-shots-Young.csv"
female_csv = "0-shots-Middle.csv"
amb_csv = "0-shots-Old.csv"
baseline_csv = "0-shots-Baseline.csv"

In [5]:
baseline_df = pd.read_csv(os.path.join(base_csv_pth, baseline_csv)).rename(columns={'Unnamed: 0': 'idx'})
male_df = pd.read_csv(os.path.join(base_csv_pth, male_csv)).rename(columns={'Unnamed: 0': 'idx'})
female_df = pd.read_csv(os.path.join(base_csv_pth, female_csv)).rename(columns={'Unnamed: 0': 'idx'})
amb_df = pd.read_csv(os.path.join(base_csv_pth, amb_csv)).rename(columns={'Unnamed: 0': 'idx'})

In [6]:
baseline_df

,idx,sentences,choices,scores
0,0,\nQ: Tracy didn't go home that evening and res...,"['Make a new plan', 'Go home and see Riley', '...",0
1,1,\nQ: Sydney walked past a homeless woman askin...,"['Sympathetic', 'Like a person who was unable ...",0
2,2,\nQ: Sasha protected the patients' rights by m...,"['Write new laws', 'Get petitions signed', 'Li...",1
3,3,\nQ: Jordan was in charge of taking the food o...,['Horrible that he let his friends down on the...,0
4,4,\nQ: Kendall opened their mouth to speak and w...,"['A very quiet person', 'A very passive person...",0
...,...,...,...,...
1949,1949,\nQ: Bailey destroyed the enemy's bunker and w...,"['Spy on enemy troop movements', 'Give the ene...",2
1950,1950,\nQ: Kai told Kendall the reason about the acc...,"['More at ease', 'Like they got something off ...",0
1951,1951,\nQ: Kai improved Jan's picture and she loved ...,"['Wanted to be helpful', 'Wanted to Jan to be ...",2
1952,1952,\nQ: Tracy's kids wanted ice cream so Aubrey f...,"['Tell her kids to say thank you', 'Get ice cr...",1


In [11]:
# misclassifications male
merged_df = pd.merge(baseline_df, male_df, on=['idx'], suffixes=('_df1', '_df2'))
merged_df['sentences_df1'] = merged_df['sentences_df1'].apply(lambda x: x.split("\n")[1].split(":")[1])
merged_df['sentences_df2'] = merged_df['sentences_df2'].apply(lambda x: x.split("\n")[1].split(":")[1])

merged_df = pd.merge(merged_df, labels_df[["idx", "targets"]], on=['idx'])

# print(merged_df.head())
# Select rows where scores in the two dataframes are not equal
# unequal_rows = merged_df[merged_df['scores_df1'] != merged_df['scores_df2']]

unequal_rows = merged_df

unequal_rows.head()
# Display the result
unequal_rows.to_csv(os.path.join(base_csv_pth, "misclassifications", "base-young-all.csv"), index=False)

In [12]:
# misclassifications female
merged_df = pd.merge(baseline_df, female_df, on=['idx'], suffixes=('_df1', '_df2'))
merged_df['sentences_df1'] = merged_df['sentences_df1'].apply(lambda x: x.split("\n")[1].split(":")[1])
merged_df['sentences_df2'] = merged_df['sentences_df2'].apply(lambda x: x.split("\n")[1].split(":")[1])

merged_df = pd.merge(merged_df, labels_df[["idx", "targets"]], on=['idx'])
# print(merged_df.head())
# Select rows where scores in the two dataframes are not equal
# unequal_rows = merged_df[merged_df['scores_df1'] != merged_df['scores_df2']]

unequal_rows = merged_df

unequal_rows.to_csv(os.path.join(base_csv_pth, "misclassifications", "base-middle-all.csv"), index=False)
# unequal_rows.head()

In [13]:
# misclassifications ambiguous
merged_df = pd.merge(baseline_df, amb_df, on=['idx'], suffixes=('_df1', '_df2'))
merged_df['sentences_df1'] = merged_df['sentences_df1'].apply(lambda x: x.split("\n")[1].split(":")[1])
merged_df['sentences_df2'] = merged_df['sentences_df2'].apply(lambda x: x.split("\n")[1].split(":")[1])

merged_df = pd.merge(merged_df, labels_df[["idx", "targets"]], on=['idx'])
# print(merged_df.head())
# Select rows where scores in the two dataframes are not equal
# unequal_rows = merged_df[merged_df['scores_df1'] != merged_df['scores_df2']]
unequal_rows = merged_df

unequal_rows.to_csv(os.path.join(base_csv_pth, "misclassifications", "base-old-all.csv"), index=False)

In [33]:
male_mis = pd.read_csv(os.path.join(base_csv_pth, "misclassifications", "base-male.csv"))
female_mis = pd.read_csv(os.path.join(base_csv_pth, "misclassifications", "base-female.csv"))

In [34]:
male_mis

,idx,sentences_df1,choices_df1,scores_df1,sentences_df2,choices_df2,scores_df2,targets,B==T,M==T
0,0,Tracy didn't go home that evening and resiste...,"['Make a new plan', 'Go home and see Riley', '...",0,Tracy skipped home that evening and fended of...,"['Make a new plan', 'Go home and see Riley', '...",2,2,False,True
1,6,Kendall's dog was overweight so they walked i...,"['Because it was unhealthy', 'Start an exercis...",0,Kendall's pooch was packing some extra pounds...,"['Because it was unhealthy', 'Start an exercis...",1,1,False,True
2,12,Carson was excited to wake up to attend schoo...,"['Take the big test', 'Just say hello to frien...",2,Carson was stoked to rise and shine for anoth...,"['Take the big test', 'Just say hello to frien...",0,1,False,False
3,13,Taylor proved Carson's point about who was th...,"['Be good at wrestling', 'Bored', 'Good']",1,Taylor solidified Carson's argument that he w...,"['Be good at wrestling', 'Bored', 'Good']",0,0,False,True
4,14,Sydney went trick or treating and the others ...,"['Go home', 'Did this to get candy', 'Get candy']",0,Sydney hit the streets for some trick-or-trea...,"['Go home', 'Did this to get candy', 'Get candy']",2,2,False,True
...,...,...,...,...,...,...,...,...,...,...
352,1936,Riley was competing in a gymnastics meet. Ril...,"['Competitive', 'Upset', 'Regretful']",1,Riley was giving it his all in the gymnastics...,"['Competitive', 'Upset', 'Regretful']",0,1,True,False
353,1946,Jordan was very hungry so he went to a restau...,"['Full', 'A foodie', 'Satisfied']",2,"Jordan was famished, so he headed to the near...","['Full', 'A foodie', 'Satisfied']",1,1,False,True
354,1949,Bailey destroyed the enemy's bunker and won t...,"['Spy on enemy troop movements', 'Give the ene...",2,Bailey single-handedly took out the enemy's h...,"['Spy on enemy troop movements', 'Give the ene...",0,0,False,True
355,1950,Kai told Kendall the reason about the acciden...,"['More at ease', 'Like they got something off ...",0,Kai explained to Kendall the reasons behind t...,"['More at ease', 'Like they got something off ...",1,1,False,True


In [35]:
male_mis = male_mis[male_mis["B==T"]==True][['idx', 'sentences_df1', 'sentences_df2']]
male_mis = male_mis.rename(columns={"sentences_df1":"original"})
female_mis = female_mis[female_mis["B==T"]==True][['idx', 'sentences_df1', 'sentences_df2']]
female_mis = female_mis.rename(columns={"sentences_df1":"original"})

In [36]:
male_mis

,idx,original,sentences_df2
5,35,Robin stayed outside and tried to finish the ...,Robin played a fierce game of volleyball with...
7,44,Robin shut Riley's eyes to the fact that he d...,Robin cleverly obscured Riley's view of the e...
8,47,Austin was taking a test and found it difficu...,Austin was really struggling with the test at...
15,85,Alex grew closer to their significant other a...,Alex strengthened his bond with his partner a...
16,98,Taylor taught math in the schools and got a l...,Taylor impressed everyone with his innovative...
...,...,...,...
346,1897,Jan wasted all their money when she went gamb...,John lost a fortune at the casino last night....
347,1899,Remy was caught between two groups of protest...,"Remy found himself in a tense situation, sand..."
348,1907,Riley wanted to show off their cooking skills...,Riley impressed everyone with their culinary ...
352,1936,Riley was competing in a gymnastics meet. Ril...,Riley was giving it his all in the gymnastics...


In [38]:
m_f_common = pd.merge(male_mis, female_mis, how='inner', on=['idx'], suffixes=('_m', '_f'))

In [39]:
m_f_common

,idx,original_m,sentences_df2_m,original_f,sentences_df2_f
0,44,Robin shut Riley's eyes to the fact that he d...,Robin cleverly obscured Riley's view of the e...,Robin shut Riley's eyes to the fact that he d...,Robin tactfully covered Riley's eyes to spare...
1,47,Austin was taking a test and found it difficu...,Austin was really struggling with the test at...,Austin was taking a test and found it difficu...,"At first, Austin found the test to be quite c..."
2,109,Austin built a mall on the site where the rui...,Austin developed a shopping center on the lan...,Austin built a mall on the site where the rui...,The city of Austin constructed a modern mall ...
3,141,Jan lit a fire under her coworkers because th...,Jan lit a fire under her colleagues' butts be...,Jan lit a fire under her coworkers because th...,"Jan ignited a spark in her teammates, inspiri..."
4,142,Alex rendered Sasha's service and was pleased...,Alex was impressed by Sasha's thoroughness in...,Alex rendered Sasha's service and was pleased...,Sasha's dedication to her work and attention ...
...,...,...,...,...,...
67,1809,Alex made Casey escape jail by blowing a hole...,Alex rescued Casey from prison by blasting a ...,Alex made Casey escape jail by blowing a hole...,"With Alex's daring escape plan, they were abl..."
68,1829,Aubrey fulfilled Alex's requests. What will A...,Aubrey catered to Alex's needs and desires. ...,Aubrey fulfilled Alex's requests. What will A...,Aubrey went above and beyond to fulfill Alex'...
69,1850,Skylar joined Sasha's ranks as one of the onl...,Skylar impressively became one of the few emp...,Skylar joined Sasha's ranks as one of the onl...,Skylar proved to be an exceptional employee b...
70,1899,Remy was caught between two groups of protest...,"Remy found himself in a tense situation, sand...",Remy was caught between two groups of protest...,Remy was caught in the middle of a heated sta...


In [40]:
m_f_common.to_csv(os.path.join(base_csv_pth, "misclassifications", "m_f_common.csv"))